In [24]:
'''
앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.'''
#https://nomadcoders.co/c/gpt-challenge/lobby -> 제출 링크
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True
)

examples = [
    {
        "movie_name": "저스티스리그",
        "answer": """
        🦸‍♂️🦸‍♀🌍"""
    },
    {
        "movie_name": "아이언맨3",
        "answer": """
        🤖💥🦸‍♂️"""
    },
    {
        "movie_name": "대부",
        "answer": """
        👨‍👨‍👦🔫🍝"""
    },
    {
        "movie_name": "탑건",
        "answer": """
        🛩️👨‍✈️🔥"""
    }
]

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{movie_name} -> {answer}"),
    ]
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영화 제목을 입력받으면 그 영화에 대한 세 개의 이모티콘으로 대답해주는 영화 전문가야."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{movie_name}에 대해 알고 있는 이모티콘을 알려줘.")
])


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
    
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(movie_name):
    result = chain.invoke({"movie_name": movie_name})
    memory.save_context(
        {"input": movie_name},
        {"output": result.content},
    )
    print(result)

In [25]:
invoke_chain("저스티스리그")

content='🦸\u200d♂️💥🦹\u200d♀️'


In [26]:
invoke_chain("아이언맨3")

content='🔥🤖💥'


In [27]:
invoke_chain("내가 물어본 영화 제목을 말해줘")

content='🤔🎥🤖'
